# Item Similarity

In [1]:
import pandas
import pandas as pd
import numpy as np

from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

from scipy.spatial.distance import cosine
from scipy.sparse import csr_matrix


import time
import itemSimilarity as Recommenders


In [2]:
track_playlist_is_model = Recommenders.item_similarity_recommender_py()
track_user_is_model =  Recommenders.item_similarity_recommender_py()

In [3]:
import time
import pickle
import math
import pandas as pd
import numpy as np
import scipy.sparse
from sklearn import preprocessing

print("succesful import")

succesful import


In [4]:
target_playlists = pd.read_csv('datasets/target_playlists.csv', sep='\t')
target_tracks = pd.read_csv('datasets/target_tracks.csv', sep='\t')
tracks_final = pd.read_csv('datasets/tracks_final.csv', sep='\t')
playlists_final = pd.read_csv('datasets/playlists_final.csv', sep='\t')
train_final = pd.read_csv('datasets/train_final.csv', sep='\t')

print('Successfully read data')

Successfully read data


In [5]:
import gc as gc
print(gc.collect())

def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])


41


In [6]:
removed_tracks = tracks_final.query('duration == -1'); 
tracks_final = tracks_final.query('duration != -1');

In [7]:
combined = train_final.merge(tracks_final, left_on = 'track_id', right_on = 'track_id', how = 'left')

In [8]:
combined = combined[combined.track_id.isin(target_tracks.track_id)]

In [9]:
combined.head(5)

,playlist_id,track_id,artist_id,duration,playcount,album,tags
0,3271849,2801526,325531.0,157000.0,1086.0,[149604],"[3982, 170251, 189631, 237215, 237214]"
1,5616275,727878,442798.0,249000.0,1498.0,[205572],"[115752, 115355, 11056, 191177, 117800]"
5,5270369,2821391,NaN,NaN,NaN,NaN,NaN
7,7908370,2498280,3747.0,188000.0,624.0,[None],"[11242, 205245, 2474, 11056, 72354]"
9,886396,863177,155304.0,151000.0,550.0,[70155],"[70618, 70625, 43212, 117167, 112283]"


In [10]:
combined = combined[-combined['playcount'].apply(np.isnan)]


In [11]:
combined.head(5)

,playlist_id,track_id,artist_id,duration,playcount,album,tags
0,3271849,2801526,325531.0,157000.0,1086.0,[149604],"[3982, 170251, 189631, 237215, 237214]"
1,5616275,727878,442798.0,249000.0,1498.0,[205572],"[115752, 115355, 11056, 191177, 117800]"
7,7908370,2498280,3747.0,188000.0,624.0,[None],"[11242, 205245, 2474, 11056, 72354]"
9,886396,863177,155304.0,151000.0,550.0,[70155],"[70618, 70625, 43212, 117167, 112283]"
10,5758965,676462,286103.0,377000.0,2977.0,[131776],"[255208, 70618, 64978, 43212, 70625]"


In [12]:
combined['tags'] = [v[1:-1].split(',') for v in combined['tags']]

In [13]:
playlistList = combined['playlist_id']
itemList = combined['track_id']
playcount = combined['playcount']
tagList = combined['tags']


In [14]:
playlists_final.head(10)
#add owner to combined datat
combined = combined.merge(playlists_final.drop(playlists_final.columns[[0, 2, 3, 4]], axis=1), left_on = 'playlist_id', right_on = 'playlist_id', how = 'left')

In [15]:
combined.head(5)

,playlist_id,track_id,artist_id,duration,playcount,album,tags,owner
0,3271849,2801526,325531.0,157000.0,1086.0,[149604],"[3982, 170251, 189631, 237215, 237214]",44122
1,5616275,727878,442798.0,249000.0,1498.0,[205572],"[115752, 115355, 11056, 191177, 117800]",3725
2,7908370,2498280,3747.0,188000.0,624.0,[None],"[11242, 205245, 2474, 11056, 72354]",40373
3,886396,863177,155304.0,151000.0,550.0,[70155],"[70618, 70625, 43212, 117167, 112283]",27265
4,5758965,676462,286103.0,377000.0,2977.0,[131776],"[255208, 70618, 64978, 43212, 70625]",40640


In [16]:
track_playlist_is_model = Recommenders.item_similarity_recommender_py()
track_user_is_model =  Recommenders.item_similarity_recommender_py()

In [17]:
track_playlist_is_model.create(combined, 'playlist_id', 'track_id')

In [18]:
playlists = playlistList.unique()
len(playlists)

41228

In [19]:
tracksList = itemList
tracks = tracksList.unique()
len(tracks)

31436

In [20]:
playlist_id = playlists[5]
playlist_items = track_playlist_is_model.get_user_items(playlist_id)
len(playlists)

41228

In [67]:

submission_csv = open('submissions.csv'.format('csv'), 'w')
submission_csv.write('playlist_id,track_ids\n')


22

In [74]:
for pl_id in target_playlists.values:
    submission_csv.write('{},'.format(pl_id[0]))
    recs = track_playlist_is_model.recommend(pl_id[0])
    recs = recs.head(5).song
    recs = [int(i) for i in recs]
    row = ''
    for track_id in recs:
        row += ' {}'.format(track_id)
    row += '\n'
    submission_csv.write(row)
    break


No. of unique songs for the user: 32
no. of unique songs in the training set: 31436
Non zero values in cooccurence_matrix :37503


In [ ]:
a = track_playlist_is_model.recommend(target_playlists.values[100][0])